# StyleTTS2 評価音声生成

学習済みモデルを使って、共通評価文で音声を生成し、音声品質を比較します。

## 評価文

以下の3つの評価文を使用:
1. "こんにちは、今日は良い天気ですね。"
2. "人工知能の発展により、音声合成技術も進化しています。"
3. "この文章を自然な音声で読み上げます。"

## 出力

各データセットごとに以下を生成:
- `outputs/eval_audio/jvs002_{dataset_name}/eval_001.wav`
- `outputs/eval_audio/jvs002_{dataset_name}/eval_002.wav`
- `outputs/eval_audio/jvs002_{dataset_name}/eval_003.wav`


In [ ]:
import sys
import yaml
from pathlib import Path
import torch
import torchaudio
import librosa
import numpy as np
from munch import Munch
import soundfile as sf

# プロジェクトルートを取得
# プロジェクトルートを取得
# notebooks/styletts2/ から実行される場合: 2階層上がる
# notebooks/ から実行される場合: 1階層上がる
current_dir = Path.cwd()
if current_dir.name == "styletts2":
    PROJECT_ROOT = current_dir.parent.parent  # notebooks/styletts2 -> notebooks -> project_root
elif current_dir.name == "notebooks":
    PROJECT_ROOT = current_dir.parent  # notebooks -> project_root
else:
    PROJECT_ROOT = current_dir  # プロジェクトルートから直接実行
STYLETTS2_DIR = PROJECT_ROOT / "StyleTTS2"
OUTPUT_ROOT = PROJECT_ROOT / "outputs"

# StyleTTS2をパスに追加
if STYLETTS2_DIR.exists() and str(STYLETTS2_DIR) not in sys.path:
    sys.path.insert(0, str(STYLETTS2_DIR))

# データセット名のリスト
DATASETS = ["phone_min4", "feat_top10", "full100"]

# 評価文
EVAL_TEXTS = [
    "こんにちは、今日は良い天気ですね。",
    "人工知能の発展により、音声合成技術も進化しています。",
    "この文章を自然な音声で読み上げます。"
]

print("="*60)
print("評価音声生成")
print("="*60)
print(f"\nプロジェクトルート: {PROJECT_ROOT}")
print(f"StyleTTS2ディレクトリ: {STYLETTS2_DIR}")
print(f"データセット: {DATASETS}")
print(f"\n評価文数: {len(EVAL_TEXTS)}")
for i, text in enumerate(EVAL_TEXTS, 1):
    print(f"  {i}. {text}")


In [ ]:
# デバイス設定
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\nデバイス: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


In [ ]:
# StyleTTS2モジュールのインポート
from models import *
from utils import *
from text_utils import TextCleaner

textcleaner = TextCleaner()

print("✓ StyleTTS2モジュールをインポートしました")


In [ ]:
# 音素化関数
from phonemizer import phonemize

def phonemize_japanese(text: str) -> str:
    """日本語テキストをeSpeak-NGで音素化"""
    try:
        phonemized = phonemize(
            text,
            backend='espeak',
            language='ja',
            strip=True,
            preserve_punctuation=True,
            with_stress=False,
            njobs=1
        )
        return phonemized.strip()
    except Exception as e:
        print(f"音素化エラー: {e}")
        return text

print("✓ 音素化関数を定義しました")


In [ ]:
# モデル読み込み関数
def load_model(dataset_name: str):
    """
    学習済みモデルを読み込む
    
    Args:
        dataset_name: データセット名
        
    Returns:
        モデルと設定のタプル
    """
    config_path = STYLETTS2_DIR / "Configs" / f"config_jvs002_{dataset_name}.yml"
    model_dir = STYLETTS2_DIR / "Models" / f"jvs002_{dataset_name}"
    
    if not config_path.exists():
        raise FileNotFoundError(f"設定ファイルが見つかりません: {config_path}")
    if not model_dir.exists():
        raise FileNotFoundError(f"モデルディレクトリが見つかりません: {model_dir}")
    
    # 設定を読み込み
    config = yaml.safe_load(open(config_path, encoding='utf-8'))
    
    # 最新のStage 2チェックポイントを探す
    checkpoint_files = list(model_dir.glob("epoch_2nd_*.pth"))
    if not checkpoint_files:
        # Stage 1チェックポイントを探す
        checkpoint_files = list(model_dir.glob("epoch_1st_*.pth"))
    
    if not checkpoint_files:
        raise FileNotFoundError(f"チェックポイントが見つかりません: {model_dir}")
    
    checkpoint_path = sorted(checkpoint_files)[-1]  # 最新のものを使用
    print(f"  チェックポイント: {checkpoint_path.name}")
    
    # ASRモデルの読み込み
    ASR_config = config.get('ASR_config', False)
    ASR_path = config.get('ASR_path', False)
    if ASR_config and ASR_path:
        ASR_config_path = STYLETTS2_DIR / ASR_config
        ASR_model_path = STYLETTS2_DIR / ASR_path
        text_aligner = load_ASR_models(str(ASR_model_path), str(ASR_config_path))
    else:
        raise ValueError("ASR設定が見つかりません")
    
    # F0モデルの読み込み
    F0_path = config.get('F0_path', False)
    if F0_path:
        F0_model_path = STYLETTS2_DIR / F0_path
        pitch_extractor = load_F0_models(str(F0_model_path))
    else:
        raise ValueError("F0設定が見つかりません")
    
    # PLBERTの読み込み
    BERT_path = config.get('PLBERT_dir', False)
    if BERT_path:
        from Utils.PLBERT.util import load_plbert
        plbert = load_plbert(str(STYLETTS2_DIR / BERT_path))
    else:
        raise ValueError("PLBERT設定が見つかりません")
    
    # モデルの構築
    model = build_model(recursive_munch(config['model_params']), text_aligner, pitch_extractor, plbert)
    
    # チェックポイントの読み込み
    params_whole = torch.load(str(checkpoint_path), map_location='cpu')
    params = params_whole.get('net', params_whole)
    
    for key in model:
        if key in params:
            try:
                model[key].load_state_dict(params[key])
            except:
                from collections import OrderedDict
                state_dict = params[key]
                new_state_dict = OrderedDict()
                for k, v in state_dict.items():
                    name = k[7:] if k.startswith('module.') else k  # module.を削除
                    new_state_dict[name] = v
                model[key].load_state_dict(new_state_dict, strict=False)
    
    # モデルを評価モードに
    _ = [model[key].eval() for key in model]
    _ = [model[key].to(device) for key in model]
    
    return model, config

print("✓ モデル読み込み関数を定義しました")


In [ ]:
# 音声生成関数
def synthesize_speech(model, text: str, ref_audio_path: str = None):
    """
    テキストから音声を生成
    
    Args:
        model: 学習済みモデル
        text: 入力テキスト
        ref_audio_path: 参照音声ファイルパス（スタイル参照用、オプション）
        
    Returns:
        生成された音声波形（numpy配列）
    """
    # テキストを音素化
    phonemized = phonemize_japanese(text)
    
    # テキストをトークン化
    tokens = textcleaner(phonemized)
    tokens.insert(0, 0)
    tokens.append(0)
    tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)
    
    # 参照音声の読み込み（スタイル参照用）
    if ref_audio_path and Path(ref_audio_path).exists():
        ref_wav, sr = librosa.load(ref_audio_path, sr=24000)
        ref_wav = torch.from_numpy(ref_wav).unsqueeze(0).to(device)
    else:
        # デフォルトのスタイル（ゼロベクトル）
        ref_wav = None
    
    # メルスペクトログラムの前処理
    to_mel = torchaudio.transforms.MelSpectrogram(
        n_mels=80, n_fft=2048, win_length=1200, hop_length=300).to(device)
    mean, std = -4, 4
    
    def preprocess(wave):
        if isinstance(wave, np.ndarray):
            wave = torch.from_numpy(wave).float()
        mel_tensor = to_mel(wave)
        mel_tensor = (torch.log(1e-5 + mel_tensor.unsqueeze(0)) - mean) / std
        return mel_tensor
    
    # スタイルベクトルの計算
    if ref_wav is not None:
        with torch.no_grad():
            ref_mel = preprocess(ref_wav.squeeze(0).cpu().numpy())
            style = model.style_encoder(ref_mel.unsqueeze(1).to(device))
    else:
        # デフォルトスタイル
        style = torch.zeros(1, 128).to(device)
    
    # Diffusion Samplerの設定
    from Modules.diffusion.sampler import DiffusionSampler, ADPM2Sampler, KarrasSchedule
    sampler = DiffusionSampler(
        model.diffusion.diffusion,
        sampler=ADPM2Sampler(),
        sigma_schedule=KarrasSchedule(sigma_min=0.0001, sigma_max=3.0, rho=9.0),
        clamp=False
    )
    
    # 音声生成
    with torch.no_grad():
        # BERT埋め込み
        bert_dur = model.bert(tokens, return_mask=False)
        
        # デュレーションモデル
        d = model.predictor.text_embedding(bert_dur)
        d = model.predictor.lstm(d)
        d = d.transpose(-1, -2)
        d = model.predictor.duration_proj(d)
        d = d.squeeze()
        
        pred_dur = torch.round(d).long()
        pred_dur = torch.clamp(pred_dur, min=1)
        
        # アライメント
        pred_aln_trg = torch.zeros(pred_dur.shape[0], pred_dur.sum()).to(device)
        c_frame = 0
        for i in range(pred_aln_trg.shape[0]):
            pred_aln_trg[i, c_frame:c_frame + pred_dur[i]] = 1
            c_frame += pred_dur[i]
        
        # スタイル拡散
        en = model.predictor.en(tokens.unsqueeze(0))
        F0_pred, N_pred = model.predictor.F0Ntrain(en, style.squeeze(1))
        
        out = sampler(noise=torch.randn((1, 128)).unsqueeze(1).to(device),
                      embedding=bert_dur,
                      embedding_scale=1.0,
                      features=F0_pred,
                      num_steps=5).squeeze(1)
        
        # デコーダー
        mel = model.decoder(out, F0_pred, N_pred, tokens.unsqueeze(0), pred_aln_trg.unsqueeze(0))
        
        # ボコーダー（mel to waveform）
        wav = model.vocoder(mel.permute(0, 1, 2))
    
    # numpy配列に変換
    wav_np = wav.squeeze().cpu().numpy()
    
    return wav_np

print("✓ 音声生成関数を定義しました")


In [ ]:
# 各データセットで評価音声を生成
print("\n" + "="*60)
print("評価音声生成開始")
print("="*60)

for dataset_name in DATASETS:
    print(f"\n{'='*60}")
    print(f"データセット: {dataset_name}")
    print(f"{'='*60}")
    
    try:
        # モデルの読み込み
        print("モデルを読み込んでいます...")
        model, config = load_model(dataset_name)
        print("✓ モデルの読み込み完了")
        
        # 出力ディレクトリの作成
        output_dir = OUTPUT_ROOT / "eval_audio" / f"jvs002_{dataset_name}"
        output_dir.mkdir(parents=True, exist_ok=True)
        
        # 参照音声の取得（データセットから1つ選ぶ）
        data_dir = STYLETTS2_DIR / "Data" / f"jvs002_{dataset_name}" / "wavs"
        ref_audio_files = list(data_dir.glob("*.wav"))
        ref_audio_path = str(ref_audio_files[0]) if ref_audio_files else None
        
        if ref_audio_path:
            print(f"参照音声: {Path(ref_audio_path).name}")
        
        # 各評価文で音声生成
        for i, text in enumerate(EVAL_TEXTS, 1):
            print(f"\n評価文 {i}/{len(EVAL_TEXTS)}: {text[:30]}...")
            
            try:
                wav = synthesize_speech(model, text, ref_audio_path)
                
                # 音声ファイルの保存
                output_path = output_dir / f"eval_{i:03d}.wav"
                sf.write(str(output_path), wav, sr=24000)
                print(f"✓ 保存: {output_path}")
                
            except Exception as e:
                print(f"✗ エラー: {e}")
                import traceback
                traceback.print_exc()
        
        print(f"\n✓ {dataset_name} の評価音声生成完了")
        print(f"出力ディレクトリ: {output_dir}")
        
    except Exception as e:
        print(f"✗ {dataset_name} の処理に失敗: {e}")
        import traceback
        traceback.print_exc()


In [ ]:
# 生成された音声ファイルの確認
print("\n" + "-"*60)
print("生成された音声ファイルの確認")
print("-"*60)

for dataset_name in DATASETS:
    output_dir = OUTPUT_ROOT / "eval_audio" / f"jvs002_{dataset_name}"
    
    if output_dir.exists():
        audio_files = sorted(output_dir.glob("eval_*.wav"))
        print(f"\n{dataset_name}: {len(audio_files)}ファイル")
        for audio_file in audio_files:
            print(f"  - {audio_file.name}")
    else:
        print(f"\n{dataset_name}: 出力ディレクトリが存在しません")


## 評価音声生成完了

各データセットの評価音声が `outputs/eval_audio/jvs002_{dataset_name}/` に保存されました。

これらの音声ファイルを使って、各データセット条件での音声品質を比較・評価できます。
